## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-03-11-50-00 +0000,wsj,Man Killed in U.K. Synagogue Attack Was Accide...,https://www.wsj.com/world/uk/u-k-officers-kill...
1,2025-10-03-11-49-12 +0000,nyt,Baltimore’s Billion-Dollar Plans to Reinvent I...,https://www.nytimes.com/2025/10/03/business/ba...
2,2025-10-03-11-39-50 +0000,nyt,Indonesia School Collapse Leaves Families Grip...,https://www.nytimes.com/2025/10/03/world/asia/...
3,2025-10-03-11-39-00 +0000,wsj,"Who Is Sarah Mullally, the Anglican Church’s F...",https://www.wsj.com/world/uk/who-is-sarah-mull...
4,2025-10-03-11-35-00 +0000,wsj,Europe to Push Back as Illegal Drone Incursion...,https://www.wsj.com/world/europe/munich-airpor...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2448/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
74,trump,35
2,attack,23
240,shutdown,22
39,manchester,19
1,synagogue,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
171,2025-10-02-22-15-00 +0000,wsj,President Trump intensified pressure on Democr...,https://www.wsj.com/politics/trump-to-meet-wit...,112
10,2025-10-03-11-21-21 +0000,wapo,One victim in Yom Kippur synagogue attack in E...,https://www.washingtonpost.com/world/2025/10/0...,96
94,2025-10-03-07-13-06 +0000,nypost,UK police name 2 victims of Manchester synagog...,https://nypost.com/2025/10/03/world-news/uk-po...,93
173,2025-10-02-22-12-02 +0000,nypost,Israeli officials rip ‘weak’ UK government for...,https://nypost.com/2025/10/02/world-news/israe...,90
131,2025-10-03-01-13-09 +0000,nyt,Trump Escalates Attacks on Democrats in a Gove...,https://www.nytimes.com/2025/10/02/us/politics...,89


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
171,112,2025-10-02-22-15-00 +0000,wsj,President Trump intensified pressure on Democr...,https://www.wsj.com/politics/trump-to-meet-wit...
10,96,2025-10-03-11-21-21 +0000,wapo,One victim in Yom Kippur synagogue attack in E...,https://www.washingtonpost.com/world/2025/10/0...
13,48,2025-10-03-11-00-46 +0000,nyt,Police Accidentally Shot and Killed a Victim D...,https://www.nytimes.com/2025/10/03/world/europ...
190,47,2025-10-02-21-25-48 +0000,nypost,Ukraine pitches plan to help US sell American ...,https://nypost.com/2025/10/02/world-news/ukrai...
141,45,2025-10-03-00-45-00 +0000,wsj,A multibillion-dollar deal to send Nvidia’s AI...,https://www.wsj.com/politics/policy/nvidia-tru...
25,41,2025-10-03-10-46-15 +0000,nypost,Sarah Mullally named the first female Archbish...,https://nypost.com/2025/10/03/world-news/churc...
181,40,2025-10-02-21-46-24 +0000,nyt,Des Moines Schools Superintendent Arrested by ...,https://www.nytimes.com/2025/10/02/us/des-moin...
130,36,2025-10-03-01-22-00 +0000,wsj,Senators on both sides of the aisle pressed th...,https://www.wsj.com/politics/national-security...
326,35,2025-10-02-13-39-46 +0000,wapo,"Israel intercepts Gaza aid flotilla, detains T...",https://www.washingtonpost.com/world/2025/10/0...
259,34,2025-10-02-17-44-40 +0000,nypost,Jane Street’s billionaire co-founder is unkemp...,https://nypost.com/2025/10/02/business/jane-st...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
